In [79]:
import pandas as pd
df = pd.read_csv('HW21.csv')
df.head()

,formula,H,He,Li,Be,B,C,N,O,F,...,Mt,Ds,Rg,Cn,Nh,Fl,Mc,Lv,Ts,Og
0,Si1 C1,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sc3 Sn1 B1,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Na1 Ga1 Te2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Nb1 In1 S2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,La1 N1,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Data Loader

In [80]:
import pandas as pd
import torch
from time import time
from torch.utils.data import Dataset, DataLoader

class Data(Dataset):
  def __init__(self):
    df = pd.read_csv('HW21.csv')
    self.Xs = torch.Tensor(df.iloc[:,1:].to_numpy())

  def __len__(self):
    return self.Xs.shape[0]

  def __getitem__(self, idx):
    x = self.Xs[idx,:]
    return idx, x

# Loading data
print('loading data...',end=''); t = time()
data = Data()
dataloader = DataLoader(data, batch_size=64, shuffle=True)
print('completed', time()-t,'sec')

torch.manual_seed(1234)

loading data...completed 0.20572662353515625 sec


### Model

In [81]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# adaopted from
# https://github.com/Zeleni9/pytorch-wgan/blob/master/models/wgan_gradient_penalty.py

class Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main_module = nn.Sequential(
            # Z latent vector 128
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,118),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return  self.main_module(x)

class Discriminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main_module = nn.Sequential(
            nn.Linear(118,128),
            nn.SELU(),
            nn.Linear(128,1))
    def forward(self, x):
        return self.main_module(x)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#build model
D = Discriminator().to(device)
G = Generator().to(device)


# Using Model

In [82]:
from time import time
import random
import numpy as np
import torch.optim as optim
from torch import autograd

def calculate_gradient_penalty(D, xr, xf):
    eta = torch.rand(xr.shape[0],1).to(device=device)
    interpolated = eta * xr + (1 - eta) * xf
    interpolated.requires_grad_()
    prob_interpolated = D(interpolated)
    gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                            grad_outputs=torch.ones_like(prob_interpolated).to(device=device),
                            create_graph=True, retain_graph=True, only_inputs=True)[0]
    grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return grad_penalty

def use_model(data_loader, D,G, criterion, Doptimizer, Goptimizer, i_iter,  name = None):
  #switch to model mode
  G.train()
  D.train()
  D_losses = []
  G_losses = []
  WasDs = []
  for idxs, xr in data_loader: # loop for each batch
    ### 1. training discriminator
    z = torch.randn(xr.shape[0], 128)
    z = z.to(device=device)
    
    xr = xr.to(device=device)
    xf = G(z).detach()
      
    Dreal_loss = D(xr).mean()
    Dfake_loss = D(xf).mean()
    gp = calculate_gradient_penalty(D, xr, xf.detach())
      
    D_loss = - Dreal_loss + Dfake_loss + 10 * gp
      
    Doptimizer.zero_grad()
    D_loss.backward()
    Doptimizer.step()

    D_losses.append(float(D_loss.item()))
    WasDs.append(abs(float((Dreal_loss - Dfake_loss).item())))

    ### 2. training generator
    z = torch.randn(xr.shape[0], 128).to(device=device)
    xf = G(z)
    G_loss = - D(xf).mean()
      
    Goptimizer.zero_grad()
    G_loss.backward()
    Goptimizer.step()
    
    G_losses.append(float(G_loss.item()))

  return np.mean(D_losses),np.mean(G_losses),np.mean(WasDs)

# Model Training and Validation

In [83]:
import torch.optim as optim

## Training
Doptimizer = optim.Adam(D.parameters(),lr=0.001)
Goptimizer = optim.Adam(G.parameters(),lr=0.001)

best_train_loss = float('inf')
for i_iter in range(1000): # epochs
  Dloss, Gloss, WD = use_model(dataloader,D,G,None,Doptimizer,Goptimizer,i_iter) # training model
  print('Train loss [%03d]: D_loss %10.2e G_loss %10.2e WasD %10.2e'%(i_iter, Dloss, Gloss, WD))
  torch.save(D.state_dict(),'DW.pth.tar') # we save the data
  torch.save(G.state_dict(),'GW.pth.tar') # we save the data
  
G.load_state_dict(torch.load('GW.pth.tar'))
G.eval()
z = torch.randn(64, 128).to(device=device)

with torch.no_grad(): # it does not compute the gradient. so it's faster
  fake_xs = G(z)


Train loss [000]: D_loss  -1.69e-01 G_loss  -6.24e-03 WasD   2.22e-01
Train loss [001]: D_loss  -1.76e-01 G_loss  -2.32e-02 WasD   1.86e-01
Train loss [002]: D_loss  -7.49e-02 G_loss   1.13e-01 WasD   8.26e-02
Train loss [003]: D_loss  -1.13e-01 G_loss   2.53e-01 WasD   1.23e-01
Train loss [004]: D_loss  -9.83e-02 G_loss   2.99e-01 WasD   1.07e-01
Train loss [005]: D_loss  -6.37e-02 G_loss   1.30e-01 WasD   7.07e-02
Train loss [006]: D_loss  -3.20e-02 G_loss  -2.03e-01 WasD   4.09e-02
Train loss [007]: D_loss  -3.14e-02 G_loss  -3.78e-01 WasD   4.05e-02
Train loss [008]: D_loss  -2.93e-02 G_loss  -4.58e-01 WasD   3.74e-02
Train loss [009]: D_loss  -3.14e-02 G_loss  -4.83e-01 WasD   4.26e-02
Train loss [010]: D_loss  -3.29e-02 G_loss  -5.68e-01 WasD   4.22e-02
Train loss [011]: D_loss  -4.02e-02 G_loss  -5.95e-01 WasD   4.93e-02
Train loss [012]: D_loss  -3.48e-02 G_loss  -5.71e-01 WasD   4.38e-02
Train loss [013]: D_loss  -3.77e-02 G_loss  -5.42e-01 WasD   5.12e-02
Train loss [014]: D_

In [65]:
z = torch.randn(64, 128)
# move input to cuda
if next(G.parameters()).is_cuda:
  z = z.to(device='cuda')
with torch.no_grad(): # it does not compute the gradient. so it's faster
  fake_xs = G(z)
import numpy as np
chemical_symbols = [
    # 0
    'X',
    # 1
    'H', 'He',
    # 2
    'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
    # 3
    'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar',
    # 4
    'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
    # 5
    'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
    'In', 'Sn', 'Sb', 'Te', 'I', 'Xe',
    # 6
    'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy',
    'Ho', 'Er', 'Tm', 'Yb', 'Lu',
    'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Po', 'At', 'Rn',
    # 7
    'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk',
    'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr',
    'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc',
    'Lv', 'Ts', 'Og']
xs = np.array(fake_xs.tolist())
xs[xs <0.01] = 0
for j in range(xs.shape[0]):
  nonzero = np.where(xs[j,:])[0]
  s = ''
  for i in nonzero:
    s += chemical_symbols[i-1] + '%.2f'%xs[j,i] +' '
  print(s)


Og0.03 H0.37 Li0.09 C0.35 F0.01 Al0.02 Sc0.05 Fe0.02 Kr0.01 Zr0.01 
Og0.05 C0.73 F0.02 Al0.04 Sc0.07 Kr0.03 Zr0.03 
Og0.07 C0.62 F0.03 Mg0.02 Al0.03 Cl0.02 Sc0.11 Kr0.08 Zr0.02 
Og0.05 C0.72 F0.02 Al0.06 Sc0.08 Kr0.02 Zr0.04 
Og0.04 C0.71 F0.04 Mg0.01 Al0.02 Cl0.02 Sc0.09 Kr0.03 Zr0.02 
Og0.04 C0.70 F0.04 Al0.02 Cl0.01 Sc0.10 Kr0.03 Zr0.02 
Og0.05 C0.68 F0.04 Mg0.01 Al0.03 Cl0.01 Sc0.09 Kr0.04 Zr0.03 
Og0.04 C0.75 F0.03 Mg0.01 Al0.03 Cl0.01 Sc0.06 Fe0.01 Kr0.02 Zr0.02 
Og0.05 B0.01 C0.65 F0.02 Mg0.02 Al0.02 Cl0.01 Sc0.08 Kr0.10 Zr0.02 
Og0.05 C0.53 F0.04 Al0.02 Cl0.04 Ar0.02 Sc0.21 Kr0.05 Zr0.02 
Og0.06 C0.69 F0.03 Mg0.02 Al0.02 Cl0.02 Sc0.08 Kr0.05 Zr0.02 
Og0.05 C0.71 F0.03 Al0.05 Sc0.06 Fe0.02 Kr0.02 Zr0.04 
Og0.05 C0.77 F0.02 Mg0.01 Al0.03 Sc0.04 Kr0.02 Zr0.03 
Og0.05 C0.66 F0.04 Mg0.01 Al0.02 Cl0.01 Sc0.11 Kr0.06 Zr0.02 
Og0.06 C0.69 F0.02 Mg0.01 Al0.03 Cl0.02 Sc0.09 Kr0.04 Zr0.03 
Og0.02 C0.66 F0.04 Al0.01 Cl0.03 Ar0.02 Sc0.13 Kr0.04 Zr0.01 
Og0.06 C0.66 F0.04 Mg0.01 Al0.03 Sc0.1

# Making Data
For your information, the code below was used to make the data

In [ ]:
from pymatgen.ext.matproj import MPRester
import numpy as np
from ase.data import chemical_symbols
from pymatgen.core import Composition
import csv

with MPRester("gv5swR0lIWsfLsGk") as m:
    docs = m.query({},{'pretty_formula':True,'icsd_ids':True})
    raw_data = [(doc['pretty_formula'],doc['icsd_ids']) for doc in docs]

new_data = {}
for f,v in raw_data:
    if len(v) != 0:
        c = Composition(f)
        new_data[str(c)] = dict(c.fractional_composition)

sym_map = {}
for i,s in enumerate(chemical_symbols):
    sym_map[s] = i -1

data = np.zeros((len(new_data),len(chemical_symbols)-1))
names = []
for i,(name,d) in enumerate(new_data.items()):
    names.append(name)
    for k,v in d.items():
        data[i,sym_map[str(k)]] = v

data_to_copy = [['formula']+chemical_symbols[1:]]
for n, r in zip(names,data):
    data_to_copy.append([n]+r.tolist())

with open('data.csv','w',newline='') as f:
    writer = csv.writer(f)
    for l in data_to_copy:
        writer.writerow(l)